In [1]:
import keras
import tensorflow as tf
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten
from tensorflow.keras.layers import MaxPooling2D
from keras.models import Sequential
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import IPython.display as ipd
%matplotlib inline
import warnings
import os
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
os.getcwd()

'C:\\Users\\vivek'

In [3]:
data_path = 'C:/Users/vivek'
os.chdir(data_path)
df = pd.read_csv('SEP-28k_labels_new.csv')
columns_to_drop = ['Show','EpId','ClipId','Start',"Stop",'Unsure','PoorAudioQuality','NoSpeech','DifficultToUnderstand']
dataset = df.drop(columns = columns_to_drop)
dataset

,Prolongation,Block,SoundRep,WordRep,Interjection,NoStutteredWords,NaturalPause,Music,Show_EpId_ClipId
0,0,0,0,0,0,3,1,0,HeStutters_0_0
1,0,0,0,0,0,3,1,0,HeStutters_0_1
2,0,0,0,0,0,3,0,0,HeStutters_0_2
3,1,0,0,0,0,2,0,0,HeStutters_0_3
4,0,0,0,0,0,3,0,0,HeStutters_0_4
...,...,...,...,...,...,...,...,...,...
28172,0,0,0,0,3,3,0,0,WomenWhoStutter_109_35
28173,1,1,3,0,1,0,0,0,WomenWhoStutter_109_36
28174,0,0,0,0,3,3,1,0,WomenWhoStutter_109_37
28175,1,0,0,0,3,2,1,0,WomenWhoStutter_109_38


In [4]:
audio_path = "D:/clips/"
new_path = os.path.join(audio_path)
os.chdir(new_path)
os.path.abspath(os.getcwd())

'D:\\clips'

In [5]:
def feature_extractor(file_name):
    y, sr = librosa.load(file_name, sr=44100, res_type='kaiser_fast')
    mfcc_features = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12)
    mfcc_scaled_features = np.mean(mfcc_features.T, axis=0)
    
    return mfcc_scaled_features

In [6]:
dataset = dataset[(dataset['Music']==0)]

In [7]:
col = ['Music']
dataset = dataset.drop(columns = col)

In [8]:
dataset

,Prolongation,Block,SoundRep,WordRep,Interjection,NoStutteredWords,NaturalPause,Show_EpId_ClipId
0,0,0,0,0,0,3,1,HeStutters_0_0
1,0,0,0,0,0,3,1,HeStutters_0_1
2,0,0,0,0,0,3,0,HeStutters_0_2
3,1,0,0,0,0,2,0,HeStutters_0_3
4,0,0,0,0,0,3,0,HeStutters_0_4
...,...,...,...,...,...,...,...,...
28172,0,0,0,0,3,3,0,WomenWhoStutter_109_35
28173,1,1,3,0,1,0,0,WomenWhoStutter_109_36
28174,0,0,0,0,3,3,1,WomenWhoStutter_109_37
28175,1,0,0,0,3,2,1,WomenWhoStutter_109_38


In [37]:
numeric_columns = dataset.columns.difference(["NoStutteredWords","Show_EpId_ClipId","Predicted_Class"])
numeric_columns

Index(['Block', 'Interjection', 'NaturalPause', 'Prolongation', 'SoundRep',
       'WordRep'],
      dtype='object')

In [39]:
from tqdm import tqdm

dataset["Predicted_Class"] = ""

for index, row in tqdm(dataset.iterrows()):
    if row["NoStutteredWords"] == 3.0:
        dataset.at[index, 'Predicted_Class'] = "NoStutteredWords"
    else:
        max_value = -1  # Initialize with a very low value
        max_class = None

        # Iterate through the class columns (Class1, Class2, ..., Class5)
        for class_column in numeric_columns:
            if row[class_column] > max_value:
                max_value = row[class_column]
                max_class = class_column
        dataset.at[index, 'Predicted_Class'] = max_class        


27781it [00:04, 5990.70it/s]


In [44]:
dataset

,Prolongation,Block,SoundRep,WordRep,Interjection,NoStutteredWords,NaturalPause,Show_EpId_ClipId,Predicted_Class
0,0,0,0,0,0,3,1,HeStutters_0_0,NoStutteredWords
1,0,0,0,0,0,3,1,HeStutters_0_1,NoStutteredWords
2,0,0,0,0,0,3,0,HeStutters_0_2,NoStutteredWords
3,1,0,0,0,0,2,0,HeStutters_0_3,Prolongation
4,0,0,0,0,0,3,0,HeStutters_0_4,NoStutteredWords
...,...,...,...,...,...,...,...,...,...
28172,0,0,0,0,3,3,0,WomenWhoStutter_109_35,NoStutteredWords
28173,1,1,3,0,1,0,0,WomenWhoStutter_109_36,SoundRep
28174,0,0,0,0,3,3,1,WomenWhoStutter_109_37,NoStutteredWords
28175,1,0,0,0,3,2,1,WomenWhoStutter_109_38,Interjection


In [43]:
extracted_features = []

for index,row in tqdm(dataset.iterrows()):
    filename = str(row['Show_EpId_ClipId'])
    duration = librosa.get_duration(filename=filename+'.wav')
    if duration != 0:
            final_class = row['Predicted_Class']
            data = feature_extractor(filename+".wav")
            extracted_features.append([data,final_class])
            


27781it [18:08, 25.53it/s] 


In [45]:
extracted_features[16373]

[array([-4.9867346e+02,  1.4851183e+02, -1.5185560e+01,  2.7246174e+01,
         3.3580891e+01,  5.6146636e+00,  2.0736704e+01,  3.7092462e+00,
        -2.4199102e+00,  1.3876469e+01,  2.0012081e+00,  3.5784130e+00,
         8.0689478e+00, -4.0390882e+00,  3.4405763e+00,  5.4892192e+00,
        -2.8387473e+00,  3.2803416e+00,  7.2290206e-01, -2.1775343e+00,
         4.4846745e+00, -4.3287192e-02, -7.2720593e-01,  3.4425199e+00,
        -1.2891451e+00, -4.8519054e-01,  1.0496652e+00, -1.3240597e+00,
         1.7382638e+00,  7.6866448e-01, -8.1345391e-01,  2.5036268e+00,
         1.4205995e-01, -9.6453804e-01,  1.4528505e+00, -5.7962620e-01,
        -5.1805991e-01,  4.8304394e-01, -1.2665305e+00, -4.7251868e-01],
       dtype=float32),
 'Block']

In [46]:
extracted_features_df = pd.DataFrame(extracted_features,columns = ['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-435.2652, 88.593636, -7.539853, 15.71825, 15...",NoStutteredWords
1,"[-428.15976, 107.88931, -8.4316635, 12.09838, ...",Block
2,"[-368.86313, 123.23408, -7.627224, 11.693373, ...",Prolongation
3,"[-382.93933, 113.41795, -26.131172, 20.281912,...",NoStutteredWords
4,"[-426.15192, 144.69386, -64.51462, 18.659634, ...",NoStutteredWords


In [2]:
extracted_features_df = pd.read_json("dataframe_for_rf.json",orient='records', lines=True)
extracted_features_df.head(30)

,feature,class
0,"[-435.2651977539, 88.5936355591, -7.539853096,...",Block
1,"[-428.1597595215, 107.8893127441, -8.431663513...",Block
2,"[-368.8631286621, 123.2340774536, -7.627223968...",Prolongation
3,"[-382.9393310547, 113.4179534912, -26.13117218...",Block
4,"[-426.1519165039, 144.693862915, -64.514617919...",Block


In [47]:
path = 'C:/Users/vivek'
os.chdir(path)
print(os.getcwd())
extracted_features_df.to_json('my_new_dataframe.json',orient='records', lines=True)

C:\Users\vivek
